In [114]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
#import random forest
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, VotingRegressor
from sklearn.model_selection import cross_val_score

In [115]:
df= pd.read_csv('../Data/Cleaned/mudita_laptops_cleaned.csv')
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Brand      303 non-null    object 
 1   Series     303 non-null    object 
 2   Price      303 non-null    float64
 3   Ram        303 non-null    float64
 4   Storage    303 non-null    float64
 5   Graphics   303 non-null    object 
 6   Processor  303 non-null    object 
dtypes: float64(3), object(4)
memory usage: 16.7+ KB


In [116]:
categorical_feature = ['Brand','Graphics','Processor']
numerical_feature = ['Ram','Storage']

scale=StandardScaler()
encode=encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

#Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scale, numerical_feature),
        ('cat', encode, categorical_feature)
    ])
#drop ZenBook ]

In [ ]:
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                              ('model', RandomForestRegressor())])
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)

models= {
    'LinearRegression': lr,
    'Random Forest': rf,
    'Gradient Boosting': gb
}

def modelling(models,X_train,y_train,X_test,y_test):
    for name, model in models.items():
        pipeline= Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"{name} - Mean Squared Error: {mse}, R^2 Score: {r2}")
        cross_val = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
        print(f"{name}Cross Validation Score: {cross_val.mean()}")

modelling(models,X_train,y_train,X_test,y_test)


#voting ensemble
voting_reg = VotingRegressor(estimators=[
    ('lr', lr), ('rf', rf), ('gb', gb)
])
voting_reg_2 = VotingRegressor(estimators=[
    ('lr', lr), ('rf', rf), ('gb', gb)
],weights=[0, 0.8, 0.5])

voting_ens={
    'Voting Ensemble': voting_reg,
    'Weighted Voting Ensemble': voting_reg_2
}

def voting_ensemble(voting_ens,X_train,y_train,X_test,y_test):
    for name, model in voting_ens.items():
        pipeline= Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"{name} - Mean Squared Error: {mse}, R^2 Score: {r2}")       



voting_ensemble(voting_ens,X_train,y_train,X_test,y_test)


LinearRegression - Mean Squared Error: 6549269833.080044, R^2 Score: 0.2843240725563725
Random Forest - Mean Squared Error: 4013402876.914558, R^2 Score: 0.5614326635874853
Gradient Boosting - Mean Squared Error: 3562120926.049644, R^2 Score: 0.610746806530948
Voting Ensemble - Mean Squared Error: 4075081213.65551, R^2 Score: 0.5546927212770765


c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
        cached_call, estimator, *args, **routed_params.get(name).score
    )
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\utils\_param_validation.py",

Voting EnsembleCross Validation Score: nan
Weighted Voting Ensemble - Mean Squared Error: 3754194895.5845776, R^2 Score: 0.5897577925205044


c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
        cached_call, estimator, *args, **routed_params.get(name).score
    )
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\utils\_param_validation.py",

Weighted Voting EnsembleCross Validation Score: nan


c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
        cached_call, estimator, *args, **routed_params.get(name).score
    )
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Roshan Acharya\Documents\Roshan\Projects\Laptop-Price-Predictior\venv\Lib\site-packages\sklearn\utils\_param_validation.py",

In [118]:
#Bootstraping



In [119]:
#Score
# train_score = pipeline.score(X_train, y_train)
# test_score= pipeline.score(X_test,y_test)
# print(f'Train Score: {train_score}')
# print(f'Test Score: {test_score}')


# #Mean Squared Error
# mse_train = mean_squared_error(y_train, pipeline.predict(X_train))
# mse_test = mean_squared_error(y_test, pipeline.predict(X_test))
# print(f'Mean Squared Error Train: {mse_train}')
# print(f'Mean Squared Error Test: {mse_test}')



In [120]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)